In [ ]:
import os
import socket
from timeit import default_timer as timer
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("").getOrCreate()

In [ ]:
country_code="US"
print('Country:',country_code)
labels=['loss', 'unemployed', 'search', 'hire', 'offer']
path_to_data='/user/spf248/twitter/data'

In [ ]:
predictions=spark.read.orc(os.path.join(path_to_data,'classification',country_code,'keywords'))
users_attributes=spark.read.orc(os.path.join(path_to_data,'users','by-country','country_code='+country_code))
users_with_timeline=spark.read.parquet(os.path.join(path_to_data,'timelines','historical','most_recent_id',country_code))

In [ ]:
users=users_attributes.join(users_with_timeline.select('user_id'),on='user_id')
users=users.withColumn('account_age', F.datediff(F.current_date(), users['created_at']))
users=users.drop('created_at')
col2func=dict((label,'max') for label in labels)
col2func.update({'created_at':'min'})
predictions=predictions.groupby('user_id').agg(col2func)
predictions=predictions.toDF(*(col.replace('max(', '').replace('min(', '').replace(')', '') for col in predictions.columns))
predictions=predictions.withColumnRenamed('created_at','oldest_tweet')
users=users.join(predictions,on='user_id')

In [ ]:
users.write.mode("overwrite").parquet(os.path.join(path_to_data,'classification',country_code,'users'))